## I. Import Lib dependences


In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr"
InteractiveShell.display_truncate_limit = 20  #

In [ ]:
from config import *
from data.reader import read_documents, read_queries, read_relevance
from data.preprocessor import preprocess_documents,create_inverted_index,compute_idf_vector,preprocess_text
from models.boolean import boolean_retrieval
from evaluation.plot import plot_precision_recall,compute_interpolated_precisions,plot_interpolated_map,evaluate_precision_recall
import matplotlib.pyplot as plt
import os
import numpy as np
from collections import Counter,OrderedDict
import re
from wordcloud import WordCloud
import pandas as pd
from numpy import dot
from numpy.linalg import norm
import math
from sklearn.preprocessing import normalize


In [ ]:
## Read data

documents = read_documents(DOCUMENTS_PATH)
queries = read_queries(QUERIES_PATH)
relevance = read_relevance(RELEVANCE_PATH)

## II. Data Overview


### Data Overview

In [ ]:
def print_documents_overview(documents, top_k=10,name="Documents"):
    num_docs = len(documents)
    lengths = [len(re.findall(r"\w+", doc)) for doc in documents]
    all_words = []

    for doc in documents:
        words = re.findall(r"\w+", doc.lower())
        all_words.extend(words)

    word_counts = Counter(all_words)
    top_words = word_counts.most_common(top_k)

    print(f"📄{name} Overview:")
    print(f"- Number of {name}: {num_docs}")
    print(f"- Sum of word: {sum(lengths)}")
    print(f"- Average word for each {name}: {np.mean(lengths):.2f}")
    print(f"- Shortest {name}: {min(lengths)} word")
    print(f"- Longest {name}: {max(lengths)} word")
    print(f"- Top {top_k} popular word:")
    for word, count in top_words:
        print(f"    {word}: {count} times")

print_documents_overview(documents)

In [ ]:
print_documents_overview(queries,name="Queries")

In [ ]:
for i in range(len(relevance)):
    print(f"Query {i+1} relevance: {relevance[i+1]}")

## III. Document Preprocessing  

#### Create Vocabulary and Process Documents

In [ ]:
processed_docs,vocab_lst = preprocess_documents(documents)

In [ ]:
for i, doc in enumerate(processed_docs):
    if len(doc) == 0:
        print(f"Document {i} is empty after preprocessing.")
        processed_docs[i] = []

##### Vocabulary

In [ ]:
print(vocab_lst)
print(len(vocab_lst))

### Wordcloud visualize

In [ ]:
all_words = ' '.join([' '.join(doc) for doc in processed_docs])

wordcloud = WordCloud(width=800, height=400, background_color='white',stopwords=None).generate(all_words)

plt.figure(figsize=(15, 7))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud for Documents')
plt.show()

## IV. Vector Space Model (VSM) Implement

### Documents indexing

#### Create Inverted Index
    Term frequency (TF_t) is defined by: the number of time term t appear in doc_id
    Inverted index
        Term -> {doc_id : TF_t}
    

In [ ]:
inverted_index = create_inverted_index(processed_docs)

print("\n📇 Inverted Index:")

for term, posting in inverted_index.items():
    print(f"{term}: {dict(posting)}")

#### Calculate idf score

    Define IDF Score: Log_10(N/n_t)
    Where N is total number of document
    n_t is total number of document that containing term "t"

In [ ]:
def print_idf_vector_pandas(idf_vector):
    df = pd.DataFrame(list(idf_vector.items()), columns=["Term", "IDF"])
    df.index.name = "Index"
    print("📐 IDF Vector Score:")
    print(df.to_string(index=True, justify='center', col_space=12, float_format="%.4f"))

idf_vector = compute_idf_vector(vocab_lst, inverted_index, len(processed_docs))
print_idf_vector_pandas(idf_vector)

#### Index Archiving with Weights
    Since the weights is defined by: TF*IDF
    Our Index become: Term -> {doc_id: weight}

    We need to sort the index hence we could search for term easier

In [ ]:
def compute_tfidf_index(index, idf):
    for term, postings in index.items():
        for doc_id, tf in postings.items():
                index[term][doc_id] = tf * idf[term]
    return index


In [ ]:
tfidf_index = compute_tfidf_index(inverted_index, idf_vector)
tfidf_index = OrderedDict(sorted(tfidf_index.items()))

In [ ]:
for term, posting in tfidf_index.items():
    print(f"{term}: {dict(posting)}")

#### Normalize tfidf_index

In [ ]:
def normalize_tfidf_index(tfidf_index, num_docs):
    doc_vectors = [[] for _ in range(num_docs + 1)]  
    terms = sorted(tfidf_index.keys())

    for term in terms:
        for doc_id in range(1, num_docs + 1):
            value = tfidf_index[term].get(doc_id, 0)
            doc_vectors[doc_id].append(value)

    norms = [0] + [math.sqrt(sum(val**2 for val in vec)) for vec in doc_vectors[1:]]

    norm_tfidf_index = OrderedDict()
    for term in terms:
        norm_tfidf_index[term] = {}
        for doc_id in range(1, num_docs + 1):
            val = tfidf_index[term].get(doc_id, 0)
            norm_val = val / norms[doc_id] if norms[doc_id] != 0 else 0
            norm_tfidf_index[term][doc_id] = norm_val

    return norm_tfidf_index


In [ ]:
norm_tfidf_index = normalize_tfidf_index(tfidf_index, len(processed_docs))

In [ ]:
for term, posting in norm_tfidf_index.items():
    print(f"{term}: {dict(posting)}")

### Query Processing


In [ ]:
def preprocess_query(query):
    return preprocess_text(query)

In [ ]:
def query_tfidf_dict(query_terms, idf_dict):
    tf_counter = Counter(query_terms)
    
    tfidf = {
        term: tf_counter[term] * idf_dict.get(term, 0)
        for term in tf_counter
    }

    vec = np.array(list(tfidf.values()))
    vec_norm = norm(vec)

    if vec_norm > 0:
        tfidf = {term: weight / vec_norm for term, weight in tfidf.items()}

    return tfidf

### Matching Query with Document

    Since both query and document are normalized, then our Cosine Similarity Formula turn out without denominator

In [ ]:
def cosine_similarity_score(query, norm_tfidf_index, idf_vector):
    query_terms = preprocess_query(query)
    query_tfidf = query_tfidf_dict(query_terms, idf_vector)

    scores = {}
    for term, weight in query_tfidf.items():
        if term in norm_tfidf_index:
            for doc_id, doc_weight in norm_tfidf_index[term].items():
                scores[doc_id] = scores.get(doc_id, 0) + weight * doc_weight


    sorted_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    return sorted_scores


###

In [ ]:
def retrieve_all_queries(list_of_queries, norm_tfidf_index, idf_vector):
    results = {}

    for qid, query in enumerate(list_of_queries, start=1):  
        scores = cosine_similarity_score(query, norm_tfidf_index, idf_vector)

        ranked_docs = [doc_id for doc_id, score in scores if score > 0]

        results[qid] = ranked_docs

    return results


In [ ]:
retrieval_results = retrieve_all_queries(queries, norm_tfidf_index, idf_vector)

#### Plot Metric Score

In [ ]:
precision, recall = evaluate_precision_recall(retrieval_results, relevance)

plot_precision_recall(precision, recall)

In [ ]:
avg_precisions_vector = compute_interpolated_precisions(retrieval_results, relevance)
plot_interpolated_map(avg_precisions_vector)

## V. LSI with boolean combined

### Latent Semantic Indexing
    Since we have inverted index with tf-idf weighted, created above, then we can create term-document matrix with tf-idf weighted respectively. Hence we will have our Matrix with shape (number of term x number of doc)

#### Create Term-document matrix

In [ ]:
terms = list(norm_tfidf_index.keys())
num_docs = len(processed_docs)

term_doc_matrix = np.zeros((len(terms), num_docs))

for i, term in enumerate(terms):
    for doc_id in range(1, num_docs + 1):
        term_doc_matrix[i, doc_id - 1] = norm_tfidf_index[term].get(doc_id, 0)


In [ ]:
print(term_doc_matrix.shape)

#### Apply Truncated SVD on Term-document matrix

In [ ]:
def find_optimal_k_from_s(singular_values, energy_threshold=0.9):
    """
        Find the optimal number of dimensions (k) based on the cumulative energy of singular values.
    """
    squared_sv = singular_values ** 2
    total_energy = np.sum(squared_sv)
    cumulative_energy = np.cumsum(squared_sv)

    for k, energy in enumerate(cumulative_energy):
        if energy / total_energy >= energy_threshold:
            return k + 1
    return len(singular_values)

In [ ]:
def apply_svd_with_flex_k(term_doc_matrix, user_defined_k=None, energy_threshold=0.9, verbose=True):
    """
    Apply SVD on term-document matrix. 
    Let:
    - User select k (user_defined_k), or
    - find k automatically base on thresh hold if user_defined_k=None

    Return: U_k, Sigma_k, Vt_k, k_used
    """
    U, s, Vt = np.linalg.svd(term_doc_matrix, full_matrices=False)

    if user_defined_k is None:
        k = find_optimal_k_from_s(s, energy_threshold)
        if verbose:
            print(f"[Auto-K] Keep {energy_threshold*100:.0f}% cumulative energy → k = {k}")
    else:
        k = user_defined_k
        if verbose:
            print(f"[Manual-K] Use k = {k} defined by user")

    U_k = U[:, :k]
    Sigma_k = np.diag(s[:k])
    Vt_k = Vt[:k, :]

    if verbose:
        print(f"\n--- SVD Component Shapes (Truncated) ---")
        print(f"U_k: {U_k.shape}, Sigma_k: {Sigma_k.shape}, Vt_k: {Vt_k.shape}")
        print("-" * 40)

    return U_k, Sigma_k, Vt_k, k

In [ ]:
U_k, Sigma_k, Vt_k, k_used = apply_svd_with_flex_k(term_doc_matrix, user_defined_k=None, energy_threshold=0.9)

#### Create Index

In [ ]:
doc_ids = [f"{i+1}" for i in range(len(documents))]

def LatentSemanticIndexing(U_k, Sigma_k, Vt_k, doc_ids):
    """
    Perform Latent Semantic Indexing (LSI) using SVD components.
    Returns the L2-normalized LSI representation of documents.
    """
    V_k_dense = Vt_k.T                      # (N_documents x k)
    doc_vectors_lsi = V_k_dense @ Sigma_k  # Shape: (N x k)

    doc_vectors_lsi_normalized = normalize(doc_vectors_lsi, axis=1) 

    # Build dict {doc_id: normalized_vector}
    document_latent_vectors = {
        doc_id: doc_vectors_lsi_normalized[i] for i, doc_id in enumerate(doc_ids)
    }

    return document_latent_vectors


### Query processing

Since we already had `idf_vector` created above, just process the query like in VSM.  
But with the LSI model, we must transform it into lower-dimensional space:  
To transform a query vector into the LSI (latent semantic indexing) space, we use the reduced SVD matrices from above.

The query vector $\mathbf{q}$ is first constructed in the original term space, then projected into the lower-dimensional LSI space using:

$\mathbf{q}_{\text{transformed}} = \Sigma_k^{-1} \cdot U_k^T \cdot \mathbf{q}$

where $U_k$ and $\Sigma_k$ are from the truncated SVD of the term-document matrix.

In [ ]:
def process_query_lsi(raw_query, idf_vector, U_k, Sigma_k):
    processed_query = preprocess_text(raw_query)
    query_tfidf = query_tfidf_dict(processed_query, idf_vector)


    processed_query = preprocess_text(raw_query)
    query_tfidf = query_tfidf_dict(processed_query, idf_vector)

    # Convert query_tfidf dict to dense vector in vocab order
    vocab = list(idf_vector.keys())
    query_tfidf_dense = np.array([query_tfidf.get(term, 0) for term in vocab])

    Sigma_k_inv = np.linalg.inv(Sigma_k) 
    query_latent_vector = query_tfidf_dense @ U_k @ Sigma_k_inv

    query_latent_vector = query_latent_vector.reshape(1, -1)  
    query_latent_vector = normalize(query_latent_vector, axis=1)  
    return query_latent_vector



### Matching Query with Document
    We would combine a Boolean layer into LSI model

In [ ]:
def LSI_with_boolean(queries, document_latent_vectors, idf_vector, U_k, Sigma_k):
    results = {}

    for qid, query in enumerate(queries, start=1):
        candidate_docs = boolean_retrieval(query, inverted_index)  
        query_latent_vector = process_query_lsi(query, idf_vector, U_k, Sigma_k)
        
        scores = {}
        for doc_id in candidate_docs:
            doc_id_str = str(doc_id)  # Ensure key is a string
            if doc_id_str in document_latent_vectors:
                score = query_latent_vector @ document_latent_vectors[doc_id_str].T
                if score > 0:
                    scores[doc_id] = score

        ranked_docs = sorted(scores.items(), key=lambda x: x[1], reverse=True)
        results[qid] = [doc_id for doc_id, _ in ranked_docs]

    return results

In [ ]:
retrieval_results_lsi = LSI_with_boolean(queries, LatentSemanticIndexing(U_k, Sigma_k, Vt_k, doc_ids), idf_vector, U_k, Sigma_k)

#### Plot Metric Score

In [ ]:
precision_1, recall_1 = evaluate_precision_recall(retrieval_results_lsi, relevance)

plot_precision_recall(precision_1, recall_1)

In [ ]:
avg_precisions_LSI = compute_interpolated_precisions(retrieval_results_lsi, relevance)
plot_interpolated_map(avg_precisions_LSI)

#### Evaluate LSI model with k - demension

In [ ]:
k_values = list(range(10, 201, 10))  
map_scores = []


In [ ]:
def analyze_map_vs_k_lsi_boolean(
    queries,
    term_doc_matrix,
    idf_vector,
    inverted_index,
    relevance_dict,
    k_list,
    doc_ids
):
    map_scores = []

    for k in k_list:
        print(f"🚀 Running LSI + Boolean Layer with k = {k}")

        U_k, Sigma_k, Vt_k, _ = apply_svd_with_flex_k(term_doc_matrix, user_defined_k=k)

        document_latent_vectors = LatentSemanticIndexing(U_k, Sigma_k, Vt_k, doc_ids)

        retrieved_docs_dict = LSI_with_boolean(
            queries, document_latent_vectors, idf_vector, U_k, Sigma_k
        )

        map_11pt = compute_interpolated_precisions(retrieved_docs_dict, relevance_dict)
        # FIX: Take mean if dict
        if isinstance(map_11pt, dict):
            mean_map = np.mean(list(map_11pt.values()))
        else:
            mean_map = map_11pt
        map_scores.append(mean_map)

    plt.figure(figsize=(8, 5))
    plt.plot(k_list, map_scores, marker='o', color='blue')
    plt.title("MAP (11-point interpolated) vs.  k - dimension in LSI")
    plt.xlabel("k - dimension")
    plt.ylabel("MAP interpolated")
    plt.grid(True)
    plt.xticks(k_list)
    plt.show()

    return None

In [ ]:
analyze_map_vs_k_lsi_boolean( queries,
    term_doc_matrix,
    idf_vector,
    inverted_index,
    relevance,
    k_list=k_values
    , doc_ids=doc_ids)
   